## Question 9.6

In [19]:
## Question 9.6
# Some code from the CNN template provided by Phudish Prateepamornkul

In [7]:
!pip install torchmetrics 
!pip install torchvision
!pip install torchsummary 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import models,transforms
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
import torchmetrics as tm

2024-03-22 15:00:33.718183: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Use CIFAR-10 dataset
def cifar10(batch_sz, path='./datasets'):
    num_classes = 10
    transform_train = transforms.Compose([
                        transforms.RandomCrop(32, padding=4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                    ])
    transform_test = transforms.Compose([
                        transforms.ToTensor(),
                    ])

    # Training dataset
    train_data = CIFAR10(root=path, train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_sz,
                                               shuffle=True, pin_memory=True)

    # Test dataset
    test_data = CIFAR10(root=path, train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_sz, shuffle=False, pin_memory=True)

    return train_loader, test_loader, num_classes


In [4]:
batch_size=64 # define the number of rows in a batch of data
train_loader, test_loader, _=cifar10(batch_size)

Files already downloaded and verified
Files already downloaded and verified


### CNN Unadjusted

In [36]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,10,5,1)
        self.conv2 = nn.Conv2d(10,50,5,1)
        self.linear1 = nn.Linear(5*5*50,100)
        self.linear2 = nn.Linear(100,10)
        self.maxpool = nn.MaxPool2d((2,2))
        self.relu = nn.ReLU()  
    def forward(self,x):
        x = self.conv1(x)
        #print(f"shape of first conv : {x.shape}")
        # x ---> (28,28,10)
        x = self.relu(x)
        x = self.maxpool(x)
        # x --> (14,14,10)
        x = self.conv2(x)
        # x --> (10,10,50)
        x = self.relu(x)
        x = self.maxpool(x)
        # x --> (5,5,50)
        x = x.view(-1,5*5*50)
        # classifier
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x
    #def get_conv_size(self):
        #print(self.conv1.in_channels)

In [37]:
# Check if GPU is available
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

net = CNN()
net = net.to(device)
lr = 0.1
mm = 0
ls_2 = []
optimizer = optim.SGD(net.parameters(),lr = lr,momentum = mm)
schdulera = optim.lr_scheduler.ExponentialLR(optimizer,0.9,verbose = True)
num_epochs = 10
loss_fn = nn.CrossEntropyLoss()
for i in range(num_epochs):
  total_loss = 0
  for batch in train_loader:
    X,y = batch[0].to(device), batch[1].to(device)
    logits = net(X)
    loss = loss_fn(logits,y)
    total_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  schdulera.step()
  ls_2.append(total_loss)

Adjusting learning rate of group 0 to 1.0000e-01.
Adjusting learning rate of group 0 to 9.0000e-02.
Adjusting learning rate of group 0 to 8.1000e-02.
Adjusting learning rate of group 0 to 7.2900e-02.
Adjusting learning rate of group 0 to 6.5610e-02.
Adjusting learning rate of group 0 to 5.9049e-02.
Adjusting learning rate of group 0 to 5.3144e-02.
Adjusting learning rate of group 0 to 4.7830e-02.
Adjusting learning rate of group 0 to 4.3047e-02.
Adjusting learning rate of group 0 to 3.8742e-02.
Adjusting learning rate of group 0 to 3.4868e-02.


In [39]:
acc = []
loss_2 = []
accuracy = tm.Accuracy(task="multiclass", num_classes=10)
total_loss = 0
for batch in test_loader:
  X,y = batch[0].to(device), batch[1].to(device)
  logits = net(X)
  yhat = torch.argmax(logits,1)
  a = accuracy(yhat.to("cpu"),y.to("cpu"))
  loss = loss_fn(logits,y)
  acc.append(a)
  loss_2.append(loss.item())
print("The mean loss value for the Convolutional Neural Networks is",np.mean(loss_2))
print("The mean accuracy value for the Convolutional Neural Networks is ",np.mean(acc))

The mean loss value for the Convolutional Neural Networks is 1.070118025609642
The mean accuracy value for the Convolutional Neural Networks is  0.6242038


The total number of bits in the input images is $3 \times 32 \times 32 \times 8 = 24576$ bits. 

Then we calculate the compression rates of each compression layer: `G-conv`. For example, from my first `nn.conv2d`, the compression ratio is $\frac{3 \times 32 \times 32}{10 \times 28\times 28} = 0.4$ and the subsequent `nn.maxpool` layer is $\frac{10 \times 28 \times 28}{10 \times 14 \times 14} = 4$. We can repeat the process and go through the other layers prior to the linear neural net, and note that ajacent $G_i$ and $G_j$ have numerators and denominators that cancel out. So in the end we should have $\frac{3 \times 32 \times 32}{5 \times 5\times 50} = 2.45$ as the final compression number ($2.45:1$).

Thus $\frac{24576}{2.45} = 10000 $ bits are passed into the linear layers. We can caluclate the MEC of the linear layer to be $(5\times 5 \times 50 + 1) \times 100 + 10 = 125110$ bits. As $125110 >> 10000$, the network currently overfits the input data.

Then we try to decrease the MEC of the final linear layer to reduce overfitting. This is done by removing the hidden layer and directly feeding the flattened vector into 10 neurons

### CNN Adjusted

In [6]:
class CNN_new(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,10,5,1)
        self.conv2 = nn.Conv2d(10,50,5,1)
        self.linear1 = nn.Linear(5*5*50,10)
        #self.linear2 = nn.Linear(100,10)
        self.maxpool = nn.MaxPool2d((2,2))
        self.relu = nn.ReLU()  
    def forward(self,x):
        # feature extractor
        x = self.conv1(x)
        #print(f"shape of first conv : {x.shape}")
        # x ---> (28,28,10)
        x = self.relu(x)
        x = self.maxpool(x)
        # x --> (14,14,10)
        x = self.conv2(x)
        # x --> (10,10,50)
        x = self.relu(x)
        x = self.maxpool(x)
        # x --> (5,5,50)
        x = x.view(-1,5*5*50)
        # classifier
        x = self.linear1(x)
        #x = self.relu(x)
        #x = self.linear2(x)
        return x
    #def get_conv_size(self):
        #print(self.conv1.in_channels)

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

net = CNN_new()
net = net.to(device)
lr = 0.1
mm = 0
ls_2 = []
optimizer = optim.SGD(net.parameters(),lr = lr,momentum = mm)
schdulera = optim.lr_scheduler.ExponentialLR(optimizer,0.9,verbose = True)
num_epochs = 10
loss_fn = nn.CrossEntropyLoss()
for i in range(num_epochs):
  total_loss = 0
  for batch in train_loader:
    X,y = batch[0].to(device), batch[1].to(device)
    logits = net(X)
    loss = loss_fn(logits,y)
    total_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  schdulera.step()
  ls_2.append(total_loss)

Adjusting learning rate of group 0 to 1.0000e-01.
Adjusting learning rate of group 0 to 9.0000e-02.
Adjusting learning rate of group 0 to 8.1000e-02.
Adjusting learning rate of group 0 to 7.2900e-02.
Adjusting learning rate of group 0 to 6.5610e-02.
Adjusting learning rate of group 0 to 5.9049e-02.
Adjusting learning rate of group 0 to 5.3144e-02.
Adjusting learning rate of group 0 to 4.7830e-02.
Adjusting learning rate of group 0 to 4.3047e-02.
Adjusting learning rate of group 0 to 3.8742e-02.
Adjusting learning rate of group 0 to 3.4868e-02.


In [8]:
acc = []
loss_2 = []
accuracy = tm.Accuracy(task="multiclass", num_classes=10)
total_loss = 0
for batch in test_loader:
  X,y = batch[0].to(device), batch[1].to(device)
  logits = net(X)
  yhat = torch.argmax(logits,1)
  a = accuracy(yhat.to("cpu"),y.to("cpu"))
  loss = loss_fn(logits,y)
  acc.append(a)
  loss_2.append(loss.item())
print("The mean loss value for the Convolutional Neural Networks is",np.mean(loss_2))
print("The mean accuracy value for the Convolutional Neural Networks is ",np.mean(acc))

The mean loss value for the Convolutional Neural Networks is 1.1188129657393049
The mean accuracy value for the Convolutional Neural Networks is  0.60828024


Now the MEC is $(5\times 5 \times 50 + 1) \times 10 = 12510$ bits, much closer to the $10000$ bits coming in. But we observe that for some reason, the mean accuracy did not improve much. 